In [1]:
import pandas as pd
from agentic.enhancer import AgenticEnhancer
import json
from termcolor import colored

In [2]:
def agentic_enhancer(original_prompt):
    
    agentic = AgenticEnhancer(original_prompt)
    agentic_answer = agentic.execute_workflow()

    original_prompt = agentic_answer.get("original_prompt")
    original_prompt_answer = agentic_answer.get("original_prompt_answer")
    original_prompt_lin_probs = agentic_answer.get("original_prompt_lin_probs")
    enhanced_prompt = agentic_answer.get("enhanced_prompt")
    final_prompt_answer = agentic_answer.get("final_prompt_answer")
    final_prompt_lin_probs = agentic_answer.get("final_prompt_lin_probs")
    
    final_result = {
        "original_prompt": original_prompt,
        "original_prompt_answer": original_prompt_answer,
        "original_prompt_lin_probs": original_prompt_lin_probs,
        "enhanced_prompt": enhanced_prompt,
        "final_prompt_answer": final_prompt_answer,
        "final_prompt_lin_probs": final_prompt_lin_probs,
    }
    
    return final_result

In [3]:
# with open('data/prompts.json', 'r') as f:
#     original_prompt = json.load(f)

# df = pd.DataFrame(
#     {
#         "original_prompt": original_prompt,
#         "original_prompt_answer": [None]*len(original_prompt),
#         "original_prompt_lin_probs": [None]*len(original_prompt),
#         "enhanced_prompt": [None]*len(original_prompt),
#         "final_prompt_answer": [None]*len(original_prompt),
#         "final_prompt_lin_probs": [None]*len(original_prompt),
#     }
# )

# display(df.head(5))




# Test with a single prompt
test_prompt = "Who is the president?"
final_result = agentic_enhancer(test_prompt)

# Create DataFrame with single result, now including evaluation metrics
df = pd.DataFrame([{
    "original_prompt": test_prompt,
    "original_prompt_answer": final_result['original_prompt_answer'],
    "original_prompt_lin_probs": final_result['original_prompt_lin_probs'],
    "enhanced_prompt": final_result['enhanced_prompt'],
    "final_prompt_answer": final_result['final_prompt_answer'],
    "final_prompt_lin_probs": final_result['final_prompt_lin_probs'],
    # New evaluation columns
    "evaluation_score": final_result.get('prompt_evaluation', {}).get('overall_score', None),
    "clarity_score": final_result.get('prompt_evaluation', {}).get('scores', {}).get('clarity', None),
    "specificity_score": final_result.get('prompt_evaluation', {}).get('scores', {}).get('specificity', None),
    "completeness_score": final_result.get('prompt_evaluation', {}).get('scores', {}).get('completeness', None),
    "structure_score": final_result.get('prompt_evaluation', {}).get('scores', {}).get('structure', None),
    "task_focus_score": final_result.get('prompt_evaluation', {}).get('scores', {}).get('task_focus', None),
    "improvement_needed": final_result.get('prompt_evaluation', {}).get('needs_improvement', None)
}])

# Display the results with evaluation metrics
display(df)






=== Workflow Progress ===
Step 1: Categorizing Prompt

Category Determined:  {query="Who is the president?", category="simple question"}

=== Workflow Progress ===
Step 1: Categorizing Prompt
Step 2: Checking for Ambiguity

Clarification Question:  which country are you asking about regarding the president?

=== Workflow Progress ===
Step 1: Categorizing Prompt
Step 2: Checking for Ambiguity
Step 3: Getting Human Feedback


Human Review Required
Prompt with History:
 ###Original Prompt###: Who is the president?
---Clarification Question: which country are you asking about regarding the president?
---User Response: usa



=== Workflow Progress ===
Step 1: Categorizing Prompt
Step 2: Checking for Ambiguity
Step 3: Getting Human Feedback
Step 4: Checking for Ambiguity

Clarification Question:  clear

=== Workflow Progress ===
Step 1: Categorizing Prompt
Step 2: Checking for Ambiguity
Step 3: Getting Human Feedback
Step 4: Checking for Ambiguity
Step 5: Rephrasing Prompt

Rephrased Prompt

,original_prompt,original_prompt_answer,original_prompt_lin_probs,enhanced_prompt,final_prompt_answer,final_prompt_lin_probs
0,Who is the president?,No original answer provided,N/A,**Role:** You are a knowledgeable political an...,"As of October 2023, the current President of t...",0.973547


In [4]:
for i, row in df.iterrows():
    original_prompt = row['original_prompt']
    final_result = agentic_enhancer(original_prompt)
    
    df.at[i, 'original_prompt_answer'] = final_result['original_prompt_answer']
    df.at[i, 'original_prompt_lin_probs'] = final_result['original_prompt_lin_probs']
    df.at[i, 'enhanced_prompt'] = final_result['enhanced_prompt']
    df.at[i, 'final_prompt_answer'] = final_result['final_prompt_answer']
    df.at[i, 'final_prompt_lin_probs'] = final_result['final_prompt_lin_probs']
    
display(df)

Category Determined:  summary
Clarification Question:  clear
Rephrased Prompt:  **Desired Output:** Provide a list of the most common methods for reducing carbon emissions.

**Context:** The user is seeking information on effective strategies or practices that can be implemented to lower carbon emissions, potentially for personal, organizational, or governmental use.

**Rephrased Prompt:**  
"What are the most common and effective methods for reducing carbon emissions, including strategies applicable to individuals, organizations, and governments?"
Enhanced Prompt:  **Role:** Environmental Policy Advisor

---

**Task:** Provide a comprehensive overview of carbon emission reduction strategies.

---

**Desired Output:**  
Please provide a detailed list of the **most common and effective methods** for reducing carbon emissions. 

**Context:** The user is seeking information on effective strategies or practices that can be implemented to lower carbon emissions, applicable to **individuals*

KeyboardInterrupt: Interrupted by user

In [6]:
df.to_csv('results/enhancement_results_1.csv', index=False)

#### --------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df = pd.read_csv('results/enhancement_results_1.csv').iloc[:, 1:]
df['original_prompt_lin_probs'] = df['original_prompt_lin_probs'].apply(lambda x: round(x, 6))

mask = df['original_prompt_lin_probs'] < 0.20
df.loc[mask, 'original_prompt_lin_probs'] = 0.50

display(df.head(5))

,original_prompt_answer,original_prompt_lin_probs,enhanced_prompt,final_prompt_answer,final_prompt_lin_probs
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


In [9]:
print(df.describe())

       original_prompt_answer  original_prompt_lin_probs  enhanced_prompt  \
count                     0.0                        0.0              0.0   
mean                      NaN                        NaN              NaN   
std                       NaN                        NaN              NaN   
min                       NaN                        NaN              NaN   
25%                       NaN                        NaN              NaN   
50%                       NaN                        NaN              NaN   
75%                       NaN                        NaN              NaN   
max                       NaN                        NaN              NaN   

       final_prompt_answer  final_prompt_lin_probs  
count                  0.0                     0.0  
mean                   NaN                     NaN  
std                    NaN                     NaN  
min                    NaN                     NaN  
25%                    NaN               

In [ ]:
# 1. Calculate and visualize average improvement percentage
def plot_improvement_distribution():
    # Calculate improvement percentage
    df['improvement_percentage'] = ((df['final_prompt_lin_probs'] - df['original_prompt_lin_probs']) 
                                  / df['original_prompt_lin_probs'] * 100)
    
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='improvement_percentage', fill=True, color='skyblue')
    plt.axvline(df['improvement_percentage'].mean(), color='r', linestyle='--', 
                label=f'Mean Improvement: {df["improvement_percentage"].mean():.1f}%')
    
    plt.title('Distribution of Prompt Enhancement Improvements')
    plt.xlabel('Improvement Percentage')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"Average improvement: {df['improvement_percentage'].mean():.1f}%")
    print(f"Maximum improvement: {df['improvement_percentage'].max():.1f}%")
    print(f"Minimum improvement: {df['improvement_percentage'].min():.1f}%")

# 2. Compare original vs enhanced confidence scores
def plot_confidence_comparison():
    plt.figure(figsize=(12, 6))
    indices = range(len(df))
    width = 0.35
    
    plt.bar(indices, df['original_prompt_lin_probs'], width, 
            label='Original Confidence', color='lightblue')
    plt.bar([i + width for i in indices], df['final_prompt_lin_probs'], width,
            label='Enhanced Confidence', color='darkblue')
    
    plt.xlabel('Prompt Index')
    plt.ylabel('Model Confidence (Linear Probability)')
    plt.title('Original vs Enhanced Prompt Confidence')
    plt.legend()
    plt.xticks([i + width/2 for i in indices], indices)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_improvement_distribution()
plot_confidence_comparison()